https://stackoverflow.com/questions/52279699/python-for-loop-millions-of-rows

In [ ]:
import numpy as np
import pandas as pd
import io
from pprint import pprint as pp

In [ ]:
s = '''
 A_D     Operator     FlightID    Terminal   TROUND_ID   tot
 A   QR  QR001   4   QR002       70
 D   DL  DL001   3   "        "  84
 D   DL  DL001   3   "        "  78
 D   VS  VS001   3   "        "  45
 A   DL  DL401   3   "        "  9
 A   DL  DL401   3   "        "  19
 A   DL  DL401   3   "        "  3
 A   DL  DL401   3   "        "  32
 A   DL  DL401   3   "        "  95
 A   DL  DL402   3   "        "  58
'''

data_aux = pd.read_table(io.StringIO(s), delim_whitespace=True)
data_aux.Terminal = data_aux.Terminal.astype(str)
data_aux.tot= data_aux.tot.astype(str)

In [ ]:
data_aux

In [ ]:
d = {'START': ['2017-03-26 16:55:00', '2017-03-26 09:30:00','2017-03-27 09:30:00','2017-10-08 15:15:00',
           '2017-03-26 06:50:00','2017-03-27 06:50:00','2017-03-29 06:50:00','2017-05-03 06:50:00',
           '2017-06-25 06:50:00','2017-03-26 07:45:00'], 'END': ['2017-10-28 16:55:00' ,'2017-06-11 09:30:00' ,
           '2017-10-28 09:30:00' ,'2017-10-22 15:15:00','2017-06-11 06:50:00' ,'2017-10-28 06:50:00', 
           '2017-04-19 06:50:00' ,'2017-10-25 06:50:00','2017-10-22 06:50:00' ,'2017-10-28 07:45:00']}    

aux_df = pd.DataFrame(data=d)
aux_df.START = pd.to_datetime(aux_df.START)
aux_df.END = pd.to_datetime(aux_df.END)


In [ ]:
aux_df

# Create the c & arr DataFrame

In [ ]:
def config_df(aux_df, data_aux):
    """
    creates c & arr DataFrame
    """
    c = pd.concat([aux_df, data_aux], axis = 1)
    c['A_D'] = c['A_D'].astype(str)
    c['Operator'] = c['Operator'].astype(str)
    c['Terminal'] = c['Terminal'].astype(str)

    c['hour'] = pd.to_datetime(c['START'], format='%H:%M').dt.time
    c['hour_aux'] = pd.to_datetime(c['START'] - pd.Timedelta(15, unit='m'), 
    format='%H:%M').dt.time
    c['start_day'] = c['START'].astype(str).str[0:10]
    c['end_day'] = c['END'].astype(str).str[0:10]
    c['x'] = c.START -  pd.to_timedelta(c.tot.astype(int), unit='m')
    c["a"] = 0
    c["Already_linked"] = np.where(c.TROUND_ID != "        ", 1 ,0)

    arr = c[c['A_D'] == 'A'].copy()
    return c, arr

In [ ]:
c, arr = config_df(aux_df=aux_df, data_aux=data_aux)

# Produce Final Output

In [ ]:
def final_output(c, arr):
    c['Already_linked'] = np.where((c.a != 0) & (c.a != 'No_link_found') & (c.A_D == 'D'), 1, c['Already_linked'])
    c.Already_linked.loc[arr.Already_linked.index] = arr.Already_linked
    c['a'] = np.where((c.Already_linked  == 0) & (c.A_D == 'D'),'No_link_found',c['a'])
    return c

# Original Code

In [ ]:
def original_way():
    """
    updates c & arr
    """
    groups = arr.groupby(['Operator', 'Terminal'])
    for row in c[(c.A_D == "D") & (c.Already_linked == 0)].itertuples():
        try:
            g = groups.get_group((row.Operator, row.Terminal))
            vb = g[(g.Already_linked==0) & (g.hour<row.hour_aux)]
            aux = (vb.START - row.x).abs().idxmin()
            c.loc[row.Index, 'a'] = vb.loc[aux].FlightID
            arr.loc[aux, 'Already_linked'] = 1
            continue
        except:
            continue

In [ ]:
# refresh c and arr to original form
c, arr = config_df(aux_df=aux_df, data_aux=data_aux)

original_way()

## Expected c & r from Original Code

In [ ]:
c_expected = c
c_expected

In [ ]:
arr_expected = arr
arr_expected

## Expected Final Output

In [ ]:
final_output(c, arr)
expected_final_c = c
expected_final_c

# New Method

## using isin

In [ ]:
def arr_hour(row, it_row):
    """
    Used inside do_g for holder3
    """
    return row.hour < it_row.hour_aux

In [ ]:
def do_g(df_test):
    """
    This is your function, but using isin
    """
    
    for it_row in df_test.itertuples():
        
        keep = {'Operator': [it_row.Operator], 'Terminal': [it_row.Terminal]}  # dict for isin combined mask
        
        holder1 = arr[list(keep)].isin(keep).all(axis=1)  # create boolean mask
        holder2 = arr.Already_linked.isin([0])  # create boolean mask
        holder3 = arr.apply(lambda row: arr_hour(row, it_row), axis=1)  # create boolean mask

        holder = holder1 & holder2 & holder3  # combine the masks
        
        holder = arr.loc[holder]

        if not holder.empty:

            aux = (holder.START - it_row.x).abs().idxmin()

            c.loc[it_row.Index, 'a'] = holder.loc[aux].FlightID  # use with apply

            arr.loc[aux, 'Already_linked'] = 1
        

## Call function

In [ ]:
# reset c and arr
c, arr = config_df(aux_df=aux_df, data_aux=data_aux)

# create the subset of the 'c' DataFrame
keep = {'A_D': ['D'], 'Already_linked': [0]}
df_test = c[c[list(keep)].isin(keep).all(axis=1)].copy()  # returns the resultant df

# call the do_g function
do_g(df_test)

# New Final Output

In [ ]:
final_output(c, arr)
new_final_c = c
new_final_c

# Compare new c & arr to expected c & arr

In [ ]:
arr == arr_expected

In [ ]:
c == c_expected

# Compare Original Final Output to New Final Output

In [ ]:
new_final_c == expected_final_c